In [92]:
import copy

class Solution:
    def containVirus(self, grid):
        def expand(grid): # 病毒扩张
            def helper(r, c):
                nonlocal seen
                stack = [(r, c)]
                while stack:
                    x, y = stack.pop()
                    for n_x, n_y in (x+1, y), (x-1, y), (x, y+1), (x, y-1):
                        if n_x < 0 or n_y < 0 or n_x >= rows or n_y >= cols:
                            continue
                        if (n_x, n_y) not in seen:
                            seen.add((n_x, n_y))
                            if grid[n_x][n_y] == 0:
                                grid[n_x][n_y] = 1
                            elif grid[n_x][n_y] == 1:
                                stack.append((n_x, n_y))
                return grid
            
            seen = set()
            for i in range(rows):
                for j in range(cols): 
                    if grid[i][j] == 1 and (i, j) not in seen: # 检测为 1 之后，需要使用helper函数进行病毒扩张
                        seen.add((i, j))
                        grid = helper(i, j)
            return grid
        
        def get_walls(grid): 
            # 首先需要找到病毒扩散最多的区域，之后对其进行抑制
            def helper(r, c):
                nonlocal seen
                virus = 0
                stack = [(r, c)]
                new_vir = set()
                while stack:
                    x, y = stack.pop()
                    for n_x, n_y in (x+1, y), (x-1, y), (x, y+1), (x, y-1):
                        if n_x < 0 or n_y < 0 or n_x >= rows or n_y >= cols:
                            continue
                        if (n_x, n_y) not in seen and (n_x, n_y) not in new_vir:
                            if grid[n_x][n_y] == 0:
                                new_vir.add((n_x, n_y))
                                virus += 1
                            elif grid[n_x][n_y] == 1:
                                seen.add((n_x, n_y))
                                stack.append((n_x, n_y))
                return virus
            
            seen = set()
            vir_max, vir_start = 0, None # 
            for i in range(rows):
                for j in range(cols):
                    if grid[i][j] == 1 and (i, j) not in seen:
                        seen.add((i, j))
                        new_vir = helper(i, j)
                        if new_vir > vir_max:
                            vir_max = new_vir
                            vir_start = (i, j)
            
            cnt = 0 # 安装墙的数量
            stack, seen = [], set()
            if vir_start: # 从 vir_start 开始安装隔离墙
                stack.append(vir_start)
                seen.add(vir_start)
            while stack:
                x, y = stack.pop()
                grid[x][y] = 2 # 表示已经隔离
                for n_x, n_y in (x+1, y), (x-1, y), (x, y+1), (x, y-1):
                    if n_x < 0 or n_y < 0 or n_x >= rows or n_y >= cols:
                        continue
                    if (n_x, n_y) not in seen:
                        if grid[n_x][n_y] == 0:
                            cnt += 1
                        elif grid[n_x][n_y] == 1:
                            seen.add((n_x, n_y))
                            stack.append((n_x, n_y))
            return cnt, grid
        
        walls = 0
        rows, cols = len(grid), len(grid[0])
        while True:
            # 在每次安装完隔离墙之后，需要对病毒进行扩张，直到最后没有变化的时候，说明完成
            wall, grid = get_walls(grid)
            walls += wall
            temp_grid = copy.deepcopy(grid)
            new_grid = expand(grid)
            if new_grid == temp_grid:
                break
            gird = new_grid
        return walls

In [111]:
import copy

class Solution:
    def containVirus(self, grid):
        def get_neibor(r, c): # 获取坐标 为 r、c的周围元素坐标
            for nr, nc in ((r-1, c), (r+1, c), (r, c-1), (r, c+1)):
                if 0 <= nr < rows and 0 <= nc < cols:
                    yield nr, nc
        
        def helper(r, c, seen, expand=False, cnt_virs=False):
            # 用于病毒扩张，以及计算病毒扩张能够影响的群体数量
            if cnt_virs:
                new_vir, virus = set(), 0
            stack = [(r, c)]
            while stack:
                x, y = stack.pop()
                for n_x, n_y in get_neibor(x, y):
                    if expand and (n_x, n_y) not in seen:
                        seen.add((n_x, n_y))
                        if grid[n_x][n_y] == 0: # 病毒扩张
                            grid[n_x][n_y] = 1
                        elif grid[n_x][n_y] == 1:
                            stack.append((n_x, n_y))
                    if cnt_virs:
                        if grid[n_x][n_y] == 0 and (n_x, n_y) not in new_vir:
                            new_vir.add((n_x, n_y))
                            virus += 1
                        elif grid[n_x][n_y] == 1 and (n_x, n_y) not in seen:
                            seen.add((n_x, n_y))
                            stack.append((n_x, n_y))
            if expand:
                return seen, grid
            if cnt_virs:
                return seen, virus
        
        def expand(grid): # 病毒扩张
            seen = set()
            for i in range(rows):
                for j in range(cols): 
                    if grid[i][j] == 1 and (i, j) not in seen: # 检测为 1 之后，需要使用helper函数进行病毒扩张
                        seen.add((i, j))
                        seen, grid = helper(i, j, seen, True)
            return grid
        
        def get_walls(grid): 
            # 首先需要找到病毒扩散最多的区域，之后对其进行抑制
            seen = set()
            vir_max, vir_start = 0, None 
            for i in range(rows):
                for j in range(cols):
                    if grid[i][j] == 1 and (i, j) not in seen:
                        seen.add((i, j))
                        seen, virus = helper(i, j, seen, False, True)
                        if virus > vir_max:
                            vir_max = virus
                            vir_start = (i, j)
            
            cnt = 0 # 安装墙的数量
            stack, seen = [], set()
            if vir_start: # 从 vir_start 开始安装隔离墙
                stack.append(vir_start)
                seen.add(vir_start)
            while stack:
                x, y = stack.pop()
                grid[x][y] = 2 # 表示已经隔离
                for n_x, n_y in get_neibor(x, y):
                    if (n_x, n_y) not in seen:
                        if grid[n_x][n_y] == 0:
                            cnt += 1
                        elif grid[n_x][n_y] == 1:
                            seen.add((n_x, n_y))
                            stack.append((n_x, n_y))
            return cnt, grid
        
        walls = 0
        rows, cols = len(grid), len(grid[0])
        while True:
            # 在每次安装完隔离墙之后，需要对病毒进行扩张，直到最后没有变化的时候，说明完成
            wall, grid = get_walls(grid)
            walls += wall
            temp_grid = copy.deepcopy(grid)
            new_grid = expand(grid)
            if new_grid == temp_grid:
                break
            gird = new_grid
        return walls

In [112]:
solution = Solution()
solution.containVirus([[0,1,1,1,1,0,1,1,0,0],
                       [0,0,0,0,0,0,1,1,0,0],
                       [0,0,0,0,0,1,0,0,0,1],
                       [0,0,1,0,0,0,0,1,0,0],
                       [1,0,0,0,0,0,1,0,0,0],
                       [0,0,1,1,0,1,0,0,1,0],
                       [0,0,0,0,0,0,0,0,0,1],
                       [0,0,0,0,0,1,1,0,0,0],
                       [0,0,0,1,0,1,0,0,0,0],
                       [0,0,1,0,0,0,0,0,1,0]])

57

In [ ]:
[[0, 1, 1, 1, 1, 0, 1, 1, 0, 0], 
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0], 
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1], 
 [0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0], 
 [0, 0, 1, 1, 0, 1, 0, 0, 1, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
 [0, 0, 0, 0, 0, 1, 1, 0, 0, 0], 
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0], 
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0]]

In [ ]:
[[0, 1, 0, 1, 1, 1, 1, 1, 1, 0], 
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 
 [0, 0, 1, 1, 1, 0, 0, 0, 1, 0], 
 [0, 0, 0, 1, 1, 0, 0, 1, 1, 0], 
 [0, 1, 0, 0, 1, 0, 1, 1, 0, 1], 
 [0, 0, 0, 1, 0, 1, 0, 1, 1, 1], 
 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0], 
 [0, 1, 0, 1, 0, 0, 0, 1, 1, 0], 
 [0, 1, 1, 0, 0, 1, 1, 0, 0, 1]]

In [69]:
class Solution:
    def containVirus(self, grid):
        R, C = len(grid), len(grid[0])
        def neighbors(r, c):
            for nr, nc in ((r-1, c), (r+1, c), (r, c-1), (r, c+1)):
                if 0 <= nr < R and 0 <= nc < C:
                    yield nr, nc

        def dfs(r, c):
            if (r, c) not in seen:
                seen.add((r, c))
                regions[-1].add((r, c))
                for nr, nc in neighbors(r, c):
                    if grid[nr][nc] == 1:
                        dfs(nr, nc)
                    elif grid[nr][nc] == 0:
                        frontiers[-1].add((nr, nc))
                        perimeters[-1] += 1

        ans = 0
        while True:
            # Find all regions, with associated frontiers and perimeters.
            seen = set()
            regions = []
            frontiers = []
            perimeters = []
            for r, row in enumerate(grid):
                for c, val in enumerate(row):
                    if val == 1 and (r, c) not in seen:
                        regions.append(set())
                        frontiers.append(set())
                        perimeters.append(0)
                        dfs(r, c)

            #If there are no regions left, break.
            if not regions: 
                break

            #Add the perimeter of the region which will infect the most squares.
            triage_index = frontiers.index(max(frontiers, key = len))
            ans += perimeters[triage_index]

            #Triage the most infectious region, and spread the rest of the regions.
            for i, reg in enumerate(regions):
                if i == triage_index:
                    for r, c in reg:
                        grid[r][c] = -1
                else:
                    for r, c in reg:
                        for nr, nc in neighbors(r, c):
                            if grid[nr][nc] == 0:
                                grid[nr][nc] = 1

        return ans

In [70]:
solution = Solution()
solution.containVirus([[0,1,0,1,1,1,1,1,1,0],
                       [0,0,0,1,0,0,0,0,0,0],
                       [0,0,1,1,1,0,0,0,1,0],
                       [0,0,0,1,1,0,0,1,1,0],
                       [0,1,0,0,1,0,1,1,0,1],
                       [0,0,0,1,0,1,0,1,1,1],
                       [0,1,0,0,1,0,0,1,1,0],
                       [0,1,0,1,0,0,0,1,1,0],
                       [0,1,1,0,0,1,1,0,0,1],
                       ])

38

In [94]:
R, C = 10, 10
def get_neibor(r, c): # 获取坐标 为 r、c的周围元素坐标
    for nr, nc in ((r-1, c), (r+1, c), (r, c-1), (r, c+1)):
        if 0 <= nr < R and 0 <= nc < C:
            yield nr, nc

In [100]:
for nr, nc in get_neibor(1, 1):
    print(nr, nc)

0 1
2 1
1 0
1 2


In [98]:
a

<generator object get_neibor at 0x000002395B687B48>